# Monday.com Timesheet
-------

# Import Libraries

In [1]:
import requests
import json
import csv
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from PIL import Image
from monday import MondayClient

# Read Config File

In [2]:
import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
api_key = config['DEFAULT']['API-Key']
timesheet_date = config['DEFAULT']['Timesheet-Date']
board_ids = config['DEFAULT']['Board-ID-List']
rate = config['DEFAULT']['Rate']
pay_code = config['DEFAULT']['Pay-Code']
subject_code = config['DEFAULT']['Subject-Code']
date = config['DEFAULT']['Current-Date']
# get section names
person_list = config.sections()

# Supporting Functions

In [3]:
def get_congif_info(person):
    surname = config[person]['Surname']
    given_names = config[person]['Given-Names']
    employee_number = config[person]['Employee-Number']
    account_code = config[person]['Account-Code']
    dob = config[person]['Date-Of-Birth']
    phone_number = config[person]['Phone-Number']
    
    return surname,given_names,employee_number,account_code,dob,phone_number

In [4]:
def create_cell(column, text, align):
    sheet["{}{}".format(column,row_num)] = text
    sheet["{}{}".format(column,row_num)].font = table_font
    sheet["{}{}".format(column,row_num)].border = thin_full_border
    if align == 1:
        sheet["{}{}".format(column,row_num)].alignment = Alignment(wrap_text = True, horizontal='center', vertical='center')
    else:
        sheet["{}{}".format(column,row_num)].alignment = Alignment(wrap_text = True, vertical='center')

In [5]:
def create_excel():
    workbook = Workbook()
    sheet = workbook.active
    
    return workbook,sheet

In [6]:
def set_column_width(sheet):
    sheet.column_dimensions['A'].width = 14
    sheet.column_dimensions['B'].width = 15.17
    sheet.column_dimensions['C'].width = 14
    sheet.column_dimensions['D'].width = 40
    sheet.column_dimensions['E'].width = 17.17
    sheet.column_dimensions['F'].width = 11.5
    sheet.column_dimensions['G'].width = 19.67
    sheet.column_dimensions['H'].width = 21.83
    sheet.column_dimensions['I'].width = 28.5
    sheet.column_dimensions['J'].width = 24.33

In [7]:
def add_logo(sheet):
    width = 88
    height = 182

    img = Image.open('timesheet_logo.png')
    img = img.resize((width,height),Image.NEAREST)
    img.save('timesheet_logo.png')

    img = openpyxl.drawing.image.Image('timesheet_logo.png')
    sheet.add_image(img,'A1')

In [8]:
def add_timesheet_info(sheet):
    sheet["B1"] = "Timesheet Sessional Academic Staff"
    sheet["B1"].font = Font(bold=True, underline="single", name='Arial', size=16)

    sheet["B2"] = "  Please  Note : "
    sheet["B2"].font = Font(bold=True, underline="single", name='Calibri (Body)', size=12)

    sheet["B3"] = """
    • This timesheet is used for sessional academic staff who work as Research Assistants, Project work or Claims that fall outside usual teaching activities logged via sessional Teaching /Marking spreadsheets.  
    • Please do not print out this T5 form, type in your timesheet details in this spreadsheet and save as “name.xlsx” file, attach it in your staff email and send to your supervisor for approval.
    • Timesheet excel format files are to be submitted from your Swinburne staff email account to your Department Administrator/Managers for appropriate approvals by no later than 5pm Tuesday.
    • Administrators/ Managers must email submit this approved Timesheets with the text in the subject: THE ATTACHED TIMESHEET IS APPROVED to fpp_sessionals@swin.edu.au by Wednesday 12 noon immediately following fortnight end date.  Please refer to paydates and deadline tab of this spreadsheet.
    • Please fill out the correct employee number and job number failure to do that will cause the incorrect payment. Please refer to Help Notes tab of this spreadsheet or contact payroll for any queries about how to fill out this timesheet. 
    """
    sheet["B3"].font = Font(name='Calibri (Body)', size=12)
    sheet["B3"].alignment = Alignment(wrap_text = True)
    sheet.row_dimensions[3].height = 125

    sheet.merge_cells('B3:J3')

In [9]:
def add_column_names(sheet):
    column_names = ["Surname", "Given Names", "Employee Number", "JOB Number", "Date", "Pay Code", "Total Hours (exc.meal break)", "Rate", "Account Code", "Subject Code"]

    for col, val in enumerate(column_names, start=1):
        sheet.cell(row=4, column=col).value = val
        sheet.cell(row=4, column=col).border = thin_full_border
        sheet.cell(row=4, column=col).font = Font(bold=True, name='Calibri (Body)', size=14)
        sheet.cell(row=4, column=col).alignment = Alignment(wrap_text = True, horizontal='center', vertical='center')

# Connect To Monday.com

In [10]:
apiUrl = "https://api.monday.com/v2"
headers = {"Authorization" : api_key}
monday = MondayClient(api_key)

# Get Board ID List

In [11]:
board_id_list = []
if board_ids:
    for b in board_ids.split(","):
        board_id_list.append(b)
else:
    query = '{ boards {id}}'
    data = {'query' : query}
    r = requests.post(url=apiUrl, json=data, headers=headers)
    for bd in r.json()["data"]["boards"]:
        board_id_list.append(bd["id"])

# Get Board.Group As Prefix For Job ID

In [12]:
group_item_dict = {}
for board_id in board_id_list:
    board_query = '{{boards (ids: {}){{name groups{{id}}}}}}'.format(board_id)
    board_data = {'query' : board_query}
    board_result = requests.post(url=apiUrl, json=board_data, headers=headers)
    
    board_name = board_result.json()["data"]["boards"][0]["name"]
    
    for g in board_result.json()["data"]["boards"][0]["groups"]:
        group_query = '{{boards (ids: {}){{groups(ids: {}) {{title items {{id}}}}}}}}'.format(board_id,g["id"])
        group_data = {'query' : group_query}
        group_result = requests.post(url=apiUrl, json=group_data, headers=headers)
        group_title = group_result.json()["data"]["boards"][0]["groups"][0]["title"]
        for i in group_result.json()["data"]["boards"][0]["groups"][0]["items"]:
            group_item_dict[i["id"]] = "{}.{}".format(board_name,group_title)

# Setup Border & Font & Color

In [13]:
thin_full_border = Border(left=Side(style='thin'), 
                     right=Side(style='thin'), 
                     top=Side(style='thin'), 
                     bottom=Side(style='thin'))

thin_top_bot_border = Border(top=Side(style='thin'), 
                     bottom=Side(style='thin'))

grey_fill = PatternFill(start_color='F2F2F2',
                   end_color='F2F2F2',
                   fill_type='solid')

table_font = Font(bold=True, name='Arial', size=12)

# Create Timesheet

In [14]:
for p in person_list:
    surname,given_names,employee_number,account_code,dob,phone_number = get_congif_info(p)
    # create new excel
    workbook,sheet = create_excel()
    
    set_column_width(sheet)
    add_logo(sheet)
    add_timesheet_info(sheet)
    add_column_names(sheet)
    
    # add tasks from Monday.com
    row_num = 5
    
    for b in board_id_list:
        query = '{{items_by_column_values (board_id :{}, column_id: "date7", column_value:"{}"){{id column_values{{text title}}}}}}'.format(b, timesheet_date)
        data = {'query' : query}
        
        r = requests.post(url=apiUrl, json=data, headers=headers) # make request
        
        for v in r.json()["data"]["items_by_column_values"]:
            job_number = "{}.{}".format(group_item_dict[v["id"]],v["id"])
            ignore = 1
            for cv in v["column_values"]:
                if cv["title"] == "Person":
                    if cv["text"] == p:
                        ignore = 0
            if ignore == 0:
                for cv in v["column_values"]:
                    if cv["title"] == "Work Day":
                        work_day = cv["text"]
                    if cv["title"] == "Time Tracking":
                        time_tracking = cv["text"]
                        if time_tracking == "":
                            time_used = 0
                        else:
                            hour = int(time_tracking.split(":")[0])
                            minute = round(int(time_tracking.split(":")[1])/60, 2)
                            time_used = hour + minute
                    if cv["title"] == "Status":
                        if cv["text"] == "Waiting for timesheet":
                            monday.items.change_item_value(b, int(v["id"]), "status", {"index": 1})

                row_list = [surname, given_names, employee_number, job_number, work_day, pay_code, time_used, rate, account_code, subject_code]
                sheet.row_dimensions[row_num].height = 25

                for col, val in enumerate(row_list, start=1):
                    sheet.cell(row=row_num, column=col).value = val
                    sheet.cell(row=row_num, column=col).border = thin_full_border
                    sheet.cell(row=row_num, column=col).fill = grey_fill

                row_num += 1
                
    # add_personal_info
    sheet["A{}".format(row_num)] = "AUTHORISATION:"
    sheet["A{}".format(row_num)].font = table_font
    sheet["A{}".format(row_num)].border = thin_top_bot_border
    sheet.merge_cells('A{}:J{}'.format(row_num,row_num))

    row_num += 2
    sheet.row_dimensions[row_num].height = 25
    sheet["A{}".format(row_num)] = "Claimant:"
    sheet["A{}".format(row_num)].font = table_font
    sheet["A{}".format(row_num)].border = thin_top_bot_border
    sheet["A{}".format(row_num)].alignment = Alignment(wrap_text = True, vertical='center')
    sheet.merge_cells('A{}:J{}'.format(row_num,row_num))

    row_num += 1
    sheet.row_dimensions[row_num].height = 25
    create_cell("A", "Name:", 0)

    create_cell("B", "{} {}".format(given_names, surname), 1)
    sheet.merge_cells('B{}:C{}'.format(row_num,row_num))

    create_cell("D", "DOB:", 0)

    create_cell("E", dob, 1)
    sheet.merge_cells('E{}:F{}'.format(row_num,row_num))

    create_cell("G", "Name:", 0)
    create_cell("H", "", 1)
    create_cell("I", "Department Name:", 0)
    create_cell("J", "", 1)

    row_num += 1
    sheet.row_dimensions[row_num].height = 25
    create_cell("A", "Phone:", 0)

    create_cell("B", phone_number, 1)
    sheet.merge_cells('B{}:C{}'.format(row_num,row_num))

    create_cell("D", "Date:", 0)

    create_cell("E", date, 1)
    sheet.merge_cells('E{}:F{}'.format(row_num,row_num))

    create_cell("G", "Phone:", 0)
    create_cell("H", "", 1)
    create_cell("I", "Date:", 0)
    create_cell("J", "", 1)
    #save changes in excel
    workbook.save(filename="{}_{}_{}.xlsx".format(given_names,surname,timesheet_date.replace('-','_')))